In [4]:
# Dataset: source code for 'requests' python libraray
dataset = open("data/requests.txt").read()

In [9]:
# sorted list of all unique characters in dataset
chars = sorted(list(set(dataset)))
vocab_size = len(chars)

# string-to-integer array
stoi = { char:i for i,char in enumerate(chars) }

# integer-to-string array
itos = { i:char for i,char in enumerate(chars) }

# lookup functions for stoi and itos arrays
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [12]:
import numpy as np

n_embd = 32

embedding_matrix = np.random.randn(vocab_size, n_embd)

print(f"Shape of the embedding matrix: {embedding_matrix.shape}")

Shape of the embedding matrix: (131, 32)
